<a href="https://colab.research.google.com/github/alexisduque/Colab/blob/main/GASRO_dytran_acc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly as plty
import seaborn as sns
import plotly.express as px
import numpy as np
from scipy.signal import butter, filtfilt
!pip install PyWavelets
import pywt
#import sktime
from pathlib import Path
try:
    import google.colab
    IN_COLAB = True
    from google.colab import drive
    drive.mount('/content/drive')
except ImportError:
    IN_COLAB = False

print("Running in Colab:", IN_COLAB)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 19.4 MB/s eta 0:00:00
Mounted at /content/drive
Running in Colab: True


In [49]:
# Data parameters
data_columns = [2, 3, 4, 5, 6, 7, 10, 11, 12]
columns_to_drop =[0, 7, 8]
sensors_positions = { 'H': '244730001972', 'GM': '244730001988', 'VL': '244730001996' }
sampling_rate = 2000 # Hz
speeds_v2 = [8, 10, 12, 14, 16]
speeds = [7, 10, 13, 15, 18]
step_duration = 60 # sec
band_pass_filter = [10, 300] # Hz
sample_count_per_step = sampling_rate * step_duration
runner_id = "GASRO"
index = {13: 968096, 7 : 1217945, 10: 1399086, 15:1612888, 18:1878611}

In [3]:
import random

def get_speeds_order():
  values = random.shuffle(speeds)  # Shuffle the values randomly

  # Now you can iterate and get each value
  for value in values:
      print(value)

In [4]:
data_path = "/content/drive/MyDrive/Dossier EOPS SMS/Stage LIBM /Movesense Validation/" if IN_COLAB else "./data/"

In [50]:
data = pd.read_csv(data_path+'GASRO/Dytran/gasro_23012025_dytran.csv', skiprows=1, header=None)
# data_cropped = data.iloc[800000:1900000]
# data_cropped = data_cropped.reset_index(drop=True)  # Reset the index if needed
data = data.drop(columns=data.columns[columns_to_drop])
data.head()

,1,2,3,4,5,6,9,10,11
0,10.839816,9.881115,10.391136,17.118689,11.709980,13.055847,12.695249,11.602747,13.922019
1,10.839816,9.881115,10.391136,17.087316,11.739804,13.055847,12.695249,11.632374,13.953051
2,10.839816,9.881115,10.391136,17.118689,11.739804,13.055847,12.665140,11.543495,13.906503
3,10.869300,9.881115,10.391136,17.087316,11.739804,13.055847,12.665140,11.573121,13.953051
4,10.839816,9.851850,10.391136,17.118689,11.739804,13.055847,12.635031,11.558308,13.890987


In [52]:
sensors = pd.read_csv(data_path+'GASRO/Dytran/gasro_23012025_dytran.csv', nrows=1, header=None)
sensors.head()

,0,1,2,3,4,5,6,7,8
0,ACC GM (56619),,,,ACC VL (60541),,,,ACC HEEL (58279)


In [ ]:
import plotly.graph_objects as go

def plot_multiple_series(data, y_columns, title, x_title, y_title, x_column=None):
    """
    Plots multiple series lines in a Plotly figure.

    Args:
        data (pd.DataFrame): The DataFrame containing the data.
        x_column (str): The name of the column to use for the x-axis.
        y_columns (list): A list of column names to use for the y-axis (series lines).
        title (str): The title of the plot.
        x_title (str): The title of the x-axis.
        y_title (str): The title of the y-axis.
    """

    fig = go.Figure()

    for column in y_columns:
        fig.add_trace(go.Scatter(x=(x_column if not None else data.index) , y=data[column],
                                 mode='lines', name=column))

    fig.update_layout(title=title,
                      xaxis_title=x_title,
                      yaxis_title=y_title)

    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="left",
        x=0
    ))

    fig.show()

In [ ]:
def slice_data():
  for key, value in index.items():
      # Slice the data
      start_index = value - sample_count_per_step
      end_index = value
      sliced_data = data.iloc[start_index:end_index]

      # Create a variable with the name of the key
      globals()[f"data_{runner_id}_{key}"] = sliced_data.reset_index()

      # Save as CSV
      filename = f"data_slice_{runner_id}_{key}.csv"
      sliced_data.to_csv(filename, index=True)

In [ ]:
for speed in speeds:
  globals()[f"data_GASRO_{speed}"] = pd.read_csv(f"{data_path}GASRO/Movesense/data_slice_GASRO_{speed}.csv")  # data_GASRO_7 = pd.read_csv(data_path + 'GASRO/Movesense/data_slice_GASRO_7.csv')

In [ ]:
for speed in speeds:
  data = globals()[f"data_GASRO_{speed}"]
  data.interpolate(method='polynomial', order=5, inplace=True)
  data.fillna(value=0.1, inplace=True)
  print(f"Speef {speed} km/h")
  print(globals()[f"data_GASRO_{speed}"].isna().sum())

In [ ]:
plot_multiple_series(data=data_GASRO_18,
                      y_columns=['Movesense_244730001972_Acc_X',
                                 'Movesense_244730001988_Acc_X',
                                 'Movesense_244730001996_Acc_X'],
                      title='Acceleration X',
                      x_title='Timestamp',
                      y_title='Acceleration X')

In [ ]:
plot_multiple_series(data=data_GASRO_13,
                      y_columns=['Movesense_244730001972_Acc_Y',
                                 'Movesense_244730001988_Acc_Y',
                                 'Movesense_244730001996_Acc_Y'],
                      title='Acceleration Y',
                      x_title='Timestamp',
                      y_title='Acceleration Y')

In [ ]:
plot_multiple_series(data=data_GASRO_10,
                      y_columns=['Movesense_244730001972_Acc_Z',
                                 'Movesense_244730001988_Acc_Z',
                                 'Movesense_244730001996_Acc_Z'],
                      title='Acceleration Z',
                      x_title='Timestamp',
                      y_title='Acceleration Z')

In [ ]:
def bandpass_filter(data, lowcut, highcut, fs, order, plot = False):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    y = filtfilt(b, a, data)
    # Calculate the frequency response
    w, h = scipy.signal.freqz(b, a, worN=8000)

    if plot == True:
      # Plot the frequency response
      plt.figure(figsize=(10, 6))  # Adjust figure size if needed
      plt.plot(0.5*fs*w/np.pi, np.abs(h), 'b')  # Convert to Hz for x-axis
      plt.title("Butterworth Bandpass Filter Frequency Response")
      plt.xlabel('Frequency [Hz]')
      plt.ylabel('Gain')
      plt.grid(True)
      plt.xlim(0, 0.5*fs)  # Limit x-axis to Nyquist frequency
      plt.show()
    return y

def compute_bandpass_filter(data_in, lowcut, highcut, fs, order=2, plot = False):
  _plot = plot
  for column in data_in.columns:
    if column.startswith('Movesense_'):
      data_in[f"{column}_filtered"] = bandpass_filter(data_in[column], lowcut, highcut, fs, order, _plot)
      _plot = False
  return data_in

In [ ]:
plot = True
for speed in speeds:
  globals()[f"data_GASRO_{speed}_filtered"] = compute_bandpass_filter(globals()[f"data_GASRO_{speed}"], band_pass_filter[0], band_pass_filter[1], sampling_rate, 5, plot)
  globals()[f"data_GASRO_{speed}_filtered"].head()
  plot = False

In [ ]:
plot_multiple_series(data=data_GASRO_15_filtered,
                      y_columns=[
                                 'Movesense_244730001988_Acc_X_filtered',
                                 'Movesense_244730001988_Acc_Y_filtered',
                                 'Movesense_244730001988_Acc_Z_filtered'],
                      title='Acceleration Filtered',
                      x_title='Timestamp',
                      y_title='Acceleration Filtered')

In [ ]:
plot_multiple_series(data=data_GASRO_13_filtered,
                      y_columns=['Movesense_244730001972_Acc_X_filtered',
                                 'Movesense_244730001988_Acc_X_filtered',
                                 'Movesense_244730001996_Acc_X_filtered'],
                      title='Acceleration Filtered',
                      x_title='Timestamp',
                      y_title='Acceleration Filtered')

In [ ]:
def compute_rms(data_in, sensors):
  for sensor_name in sensors:
    data_in[f"Movesense_{sensor_name}_RMS_filtered"] = np.sqrt(data_in[f"Movesense_{sensor_name}_Acc_X_filtered"]**2 + data_in[f"Movesense_{sensor_name}_Acc_Y_filtered"]**2 + data_in[f"Movesense_{sensor_name}_Acc_Z_filtered"]**2)
  return data_in

In [ ]:
for speed in speeds:
  globals()[f"data_GASRO_{speed}_filtered"] = compute_rms(globals()[f"data_GASRO_{speed}_filtered"], sensors_positions.values())

In [ ]:
plot_multiple_series(data=data_GASRO_10_filtered,
                      y_columns=['Movesense_244730001972_RMS_filtered',
                                 'Movesense_244730001988_RMS_filtered',
                                 'Movesense_244730001996_RMS_filtered'],
                      title='Acceleration RMS',
                      x_title='Timestamp',
                      y_title='Acceleration RMS')

In [ ]:
plot_multiple_series(data=data_GASRO_13_filtered,
                      y_columns=['Movesense_244730001972_RMS_filtered',
                                 'Movesense_244730001988_RMS_filtered',
                                 'Movesense_244730001996_RMS_filtered'],
                      title='Acceleration RMS',
                      x_title='Timestamp',
                      y_title='Acceleration RMS')

In [ ]:
import plotly.graph_objects as go

data = data_GASRO_13_filtered
# Calculate the FFT
N = len(data['Movesense_244730001988_RMS_filtered'])  # Number of samples
T = 1/sampling_rate  # Sampling period (inverse of sampling frequency)
yf = np.fft.fft(data['Movesense_244730001988_RMS_filtered'])
xf = np.fft.fftfreq(N, T)[:N//2]  # Frequency values

# Create frequency bins
bin_width = 2  # 3 Hz bins
num_bins = int(xf[-1] / bin_width) + 1
bins = np.linspace(0, xf[-1], num_bins)

# Bin the FFT data
binned_yf = np.zeros(num_bins)
for i in range(len(xf)):
    bin_index = int(xf[i] / bin_width)
    binned_yf[bin_index] += np.abs(yf[i])

# Create the plot
fig = go.Figure()
fig.add_trace(go.Bar(x=bins, y=2.0/N * binned_yf, name='FFT'))
fig.update_layout(title="FFT of Filtered Amplitude (3 Hz Bins)",
                  xaxis_title="Frequency (Hz)", yaxis_title="Magnitude",
                  xaxis_range=[0, 200])
fig.show()

In [ ]:
data_rms = data_GASRO_13_filtered

# Threashold value (m^s-2)
threashold72 = data_rms['Movesense_244730001972_RMS_filtered'].max()/2
threashold88 = data_rms['Movesense_244730001988_RMS_filtered'].max()/2
threashold96 = data_rms['Movesense_244730001996_RMS_filtered'].max()/2

# Distance btwn peaks (ms)
distance = 750
from scipy.signal import find_peaks

# Get the filtered amplitude data as a NumPy array
filtered_amplitude = data_rms['Movesense_244730001972_RMS_filtered'].values

# Find peaks
peaks72, _ = find_peaks(data_rms['Movesense_244730001972_RMS_filtered'].values, height=threashold72, distance=distance)
peaks88, _ = find_peaks(data_rms['Movesense_244730001988_RMS_filtered'].values, height=threashold88, distance=distance)
peaks96, _ = find_peaks(data_rms['Movesense_244730001996_RMS_filtered'].values, height=threashold96, distance=distance)

peak_data72 = []
peak_data88 = []
peak_data96 = []
peak_index = 0
print(f"Found {len(peaks72)} steps in #72 accelerometer time series")
print(f"Found {len(peaks88)} steps in #88 accelerometer time series")
print(f"Found {len(peaks96)} steps in #96 accelerometer time series")

for peak_index in peaks72:
    start_index = max(0, peak_index - 400)
    end_index = min(len(data), peak_index + 800)
    peak_df = data_rms.iloc[start_index:end_index]
    peak_data72.append(peak_df)
for peak_index in peaks88:
    start_index = max(0, peak_index - 400)
    end_index = min(len(data), peak_index + 800)
    peak_df = data_rms.iloc[start_index:end_index]
    peak_data88.append(peak_df)
for peak_index in peaks96:
    start_index = max(0, peak_index - 400)
    end_index = min(len(data), peak_index + 800)
    peak_df = data_rms.iloc[start_index:end_index]
    peak_data96.append(peak_df)

In [ ]:
# from sktime.alignment.dtw_python import AlignerDTW

# # Assuming data_rms is your DataFrame with columns for each time series
# # Select the columns you want to align (e.g., RMS values for different sensors)
# columns_to_align = ['Movesense_244730001972_RMS_filtered',
#                     'Movesense_244730001988_RMS_filtered']
# df1 = peak_data72[1][['relative_id', 'Movesense_244730001972_RMS_filtered']]
# df2 = peak_data72[1][['relative_id', 'Movesense_244730001988_RMS_filtered']]


# X = [df1, df2]
# # Create the AlignerDTW object
# aligner = AlignerDTW(dist_method="euclidean", step_pattern="symmetric2")

# # Fit the aligner to your data
# aligner.fit(X)

# # Get the aligned time series as a pandas DataFrame
# aligned_df = aligner.get_alignment()

In [ ]:
fig = go.Figure()
start = 40
number_of_steps = 15
step_diff_88 = -2
step_diff_96 = -2
for i, peak_df in enumerate(peak_data72[start:start+number_of_steps]):
    fig.add_trace(go.Scatter(x=peak_df['relative_id'], y=peak_df['Movesense_244730001972_RMS_filtered'],
                             mode='lines', name=f'Peak 72 {i+1+start}'))
    fig.add_trace(go.Scatter(x=peak_data88[start+i+step_diff_88]['relative_id'], y=peak_data88[start+i+step_diff_88]['Movesense_244730001988_RMS_filtered'],
                             mode='lines', name=f'Peak 88 {i+1+start}'))
    fig.add_trace(go.Scatter(x=peak_data96[start+i+step_diff_96]['relative_id'], y=peak_data96[start+i+step_diff_96]['Movesense_244730001996_RMS_filtered'],
                             mode='lines', name=f'Peak 96 {i+1+start}'))
    if i > 0: break

fig.update_layout(title='Peaks in Filtered Amplitude',
                  xaxis_title='Timestamp',
                  yaxis_title='Filtered Amplitude')
fig.show()

In [ ]:
val_max = peak_data72[start]['Movesense_244730001972_RMS_filtered'].idxmax()
t_max_72 = peak_data72[start].loc[val_max]['relative_id']
val_max = peak_data72[start]['Movesense_244730001988_RMS_filtered'].idxmax()
t_max_88 = peak_data72[start].loc[val_max]['relative_id']
val_max = peak_data72[start]['Movesense_244730001996_RMS_filtered'].idxmax()
t_max_96 = peak_data72[start].loc[val_max]['relative_id']
shift = max(t_max_72, t_max_88) - min((t_max_72, t_max_88))
print(t_max_72, t_max_88)
print(shift)
shift = max(t_max_72, t_max_96) - min((t_max_72, t_max_96))
print(t_max_72, t_max_96)

print(shift)

In [ ]:
from scipy import signal

def compute_psd(signal_data, sampling_rate, window='hann', nfft=None, detrend='constant'):
    """
    Computes the Power Spectral Density (PSD) of a signal.

    Args:
        signal_data (numpy.ndarray): The input signal data.
        sampling_rate (float): The sampling rate of the signal in Hz.
        window (str or tuple, optional): The window to use for the FFT. Defaults to 'hann'.
                                         See scipy.signal.windows for options.  Can also be a window array.
        nfft (int, optional): The length of the FFT used. If None, uses the length of the signal.
                             Padding with zeros is done if nfft is larger than the signal.
        detrend (str, optional): Method to remove linear trends from the data before FFT.
                                 'constant' removes the mean. 'linear' removes a linear trend.
                                 'none' leaves the data as is. Defaults to 'constant'.

    Returns:
        tuple: A tuple containing the frequencies and the PSD values.
               (frequencies, psd)
    """

    if nfft is None:
        nfft = len(signal_data)

    frequencies, psd = signal.welch(signal_data, sampling_rate, window=window, nfft=nfft, detrend=detrend)  # Use Welch's method for better PSD estimation

    return frequencies, psd


signal_data = peak_data72[start]['Movesense_244730001972_RMS_filtered']
signal_time = peak_data72[start]['relative_id']
# 2. Compute the PSD
frequencies, psd = compute_psd(signal_data, sampling_rate)

In [ ]:
# Assuming data_rms is already defined and contains the signal

signal_data = peak_data72[start+1]['Movesense_244730001972_RMS_filtered']

# 1. Amplitude maximale
amplitude_max = np.max(signal_data)

# 2. Calculer la PSD (Power Spectral Density)
frequencies, psd = signal.welch(signal_data, sampling_rate)

# 3. Fréquence médiane
median_psd = np.median(psd)
median_psd_index = np.where(psd == median_psd)[0][0]
median_frequency = frequencies[median_psd_index]

frequence_max = frequencies[np.argmax(psd)]
# 5. Fréquence principale (fréquence avec la PSD maximale)
frequence_max_index = np.argmax(psd)
principal_frequency = frequencies[frequence_max_index]

# 6. Intégrale de la fréquence (intégrale de la PSD)
spectrum_integral = np.trapz(psd, frequencies)

# 7. Valeur moyenne quadratique du spectre (RMS du spectre)
spectrum_rms = np.sqrt(np.mean(psd**2))
amplitude_rms = np.sqrt(np.mean(signal_data**2))


# Affichage des résultats
print("Amplitude maximale:", amplitude_max)
print("Fréquence maximale:", frequence_max)
print("Fréquence médiane:", median_frequency)
print("Fréquence principale:", principal_frequency)
print("Intégrale de la fréquence:", spectrum_integral)
print("Valeur moyenne quadratique du spectre:", spectrum_rms)
print("Valeur moyenne quadratique de l'amplitude:", amplitude_rms)

In [ ]:
# Plot with Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=frequencies, y=psd, mode='lines'))

fig.update_layout(
    title='Power Spectral Density (PSD)',
    xaxis_title='Frequency (Hz)',
    yaxis_title='PSD (dB/Hz)', # Still typically dB/Hz even though we are not explicitly converting
    #yaxis_type="log",  # Logarithmic scale for y-axis
    xaxis_range=[0, 500], # limit to Nyquist frequency
    grid=dict(rows=True, columns=True)
)

fig.show()

In [ ]:
# Calculate the wavelet transform
scales = np.arange(1, 100)  # Adjust scales as needed
wavelet = 'morl'  # Choose a wavelet (e.g., 'morl', 'db4', etc.)
coefficients, frequencies = pywt.cwt(signal_data, scales, wavelet, sampling_period=T)

In [ ]:
# Create the 3D plot
# https://plotly.com/python/builtin-colorscales/
fig = go.Figure(go.Surface(z=np.abs(coefficients), x=signal_time, y=frequencies, colorscale='sunset'))
fig.update_layout(title='Wavelet Transform (3D)', scene=dict(
                    xaxis_title='Time',
                    yaxis_title='Frequency (Hz)',
                    yaxis_range=[0, 150],
                    zaxis_title='Magnitude'))
fig.show()

In [ ]:
# Create the time-frequency plot
plt.figure(figsize=(10, 6))  # Adjust figure size as needed
plt.imshow(np.abs(coefficients), extent=[0, len(signal_data), frequencies[-1], frequencies[0]],
           aspect='auto', cmap='RdPu_r', interpolation='nearest')
plt.colorbar(label='Magnitude')
plt.xlabel('Time')
plt.ylabel('Frequency (Hz)')
plt.title('Wavelet Transform (Time-Frequency)')
plt.show()